In [ ]:
import random
from pprint import pprint

import ipywidgets as widgets
import pandas as pd
import torch
import whisper
from IPython.display import clear_output, display

from unified_desktop import RESOURCES_DIR
from unified_desktop.pipelines import UDIntentClassification, UDSpeechRecognizer

pd.options.display.max_colwidth = 100
%load_ext autoreload
%autoreload 2

In [ ]:
# Cuda dropdown

cuda_options = [torch.device("cuda", idx) for idx in range(torch.cuda.device_count())]
device_dropdown = widgets.Dropdown(
    options=["cpu"] + cuda_options,
    value="cpu",
    description="Device:",
)

## Automatic Speech Recognition (ASR)

In [ ]:
def update_asr_obj(change):
    global asrObj
    clear_output()
    display(model_dropdown_ASR, device_dropdown)
    asrObj = UDSpeechRecognizer(
        name=model_dropdown_ASR.value, device=device_dropdown.value
    )
    print(
        f"Loaded model: {model_dropdown_ASR.value} on device: {device_dropdown.value}"
    )


# Dropdown for OpenAI Whisper models
model_dropdown_ASR = widgets.Dropdown(
    options=whisper.available_models(),
    value="tiny.en",
    description="ModelName:",
)

# Attach the update function to the dropdown
model_dropdown_ASR.observe(update_asr_obj, names="value")
device_dropdown.observe(update_asr_obj, names="value")

# Display the widgets and initialize asrObj
display(model_dropdown_ASR, device_dropdown)
asrObj = UDSpeechRecognizer(name=model_dropdown_ASR.value, device=device_dropdown.value)

In [ ]:
# Audio file to transcribe
audio_file = (
    RESOURCES_DIR
    / "call-center-sample-en_US"
    / "en_US_7a4f56d7-9aca-4ed5-96b9-9c9c36b8a3ac.wav"
)
# Use fp16 if on CUDA, else fp32
fp16 = device_dropdown.value in cuda_options

# Transcribe the audio file
transcribed_text = asrObj(audio_file, verbose=True, fp16=fp16)
pprint(transcribed_text)

# Intent Detection

In [ ]:
available_models = ["vineetsharma/customer-support-intent-albert", "albert-base-v2"]


def model_obj(change):
    global intentObj
    clear_output()
    display(model_dropdown_intent, device_dropdown)
    intentObj = UDIntentClassification(
        available_models,
        name=model_dropdown_intent.value,
        device=device_dropdown.value,
    )
    print(
        f"Loaded model: {model_dropdown_intent.value} on device: {device_dropdown.value}"
    )


# Dropdown for intentDetection transformer models. More models will be added
model_dropdown_intent = widgets.Dropdown(
    options=available_models,
    value="vineetsharma/customer-support-intent-albert",
    # value = "albert-base-v2",
    description="ModelName:",
)

# Attach the update function to the dropdown
model_dropdown_intent.observe(model_obj, names="value")
device_dropdown.observe(model_obj, names="value")

# Display the widgets and initialize intentObj
display(model_dropdown_intent, device_dropdown)
intentObj = UDIntentClassification(
    name=model_dropdown_intent.value,
    device=device_dropdown.value,
)

In [ ]:
# Input text for intent detection
excel_file = RESOURCES_DIR / "input_text" / "textqueries.xlsx"

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file)
# Specify the row number (k) you want to select (replace with the desired row number)
num_rows = df.shape[0]
random_row_index = random.randint(0, num_rows - 1)
selected_row = df.iloc[random_row_index]
# Convert the selected row to a string
intent_input = selected_row.to_string(index=False)

# Display the selected row as a string
print(f"inpute query:")
print(intent_input, "\n")

# returns top_k predictions
top_k = 2

# Perform intent detection
intent_results = intentObj(intent_input, top_k)
print(intent_results)
# Print the top-k predictions and their probabilities for the intent detection
print(f"Top-{top_k} Intent Predictions:")
print("------------------------------------------------------")
print("| Intent                    | Probability")
print("------------------------------------------------------")
for items in intent_results:
    print(f"| {items['label']:<25} | {items['score']:.4f}")
print("------------------------------------------------------")